<a href="https://colab.research.google.com/github/DataSphereX/26-Weeks-Of-Data-Science/blob/master/Janus_demo_sarvesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/deepseek-ai/Janus.git


Cloning into 'Janus'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 121 (delta 51), reused 36 (delta 36), pack-reused 47 (from 2)
Receiving objects: 100% (121/121), 7.19 MiB | 30.42 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
%cd Janus
!pip install -e .
!pip install flash-attn

/content/Janus
Obtaining file:///content/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

In [5]:
model_path = "deepseek-ai/Janus-1.3B"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

Some kwargs in processor config are unused and will not have any effect: add_special_token, sft_format, num_image_tokens, ignore_id, mask_prompt, image_tag. 


Add image tag = <image_placeholder> to the tokenizer


In [6]:
# vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
#     model_path, trust_remote_code=True,
#     attn_implementation="eager",
#     torch_dtype=torch.float16,  # Ensure model uses float16
#     device_map="auto"  # Automatically assigns to GPU if available
# )
# vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

In [18]:
import os
import torch
from transformers import AutoModelForCausalLM

# Disable FlashAttention explicitly
os.environ["FLASHATTN_FORCE_DISABLE"] = "1"

# Load the model and ensure it stays on the GPU
vl_gpt = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    attn_implementation="eager",  # Use eager execution (safe for T4)
    torch_dtype=torch.float16,  # Use float16 for memory efficiency
    device_map="cuda"  # Force model to load fully onto GPU
)

# Ensure FlashAttention is disabled in the model config
if hasattr(vl_gpt.config, "use_flash_attention_2"):
    vl_gpt.config.use_flash_attention_2 = False

if hasattr(vl_gpt.config, "use_flash_attention"):
    vl_gpt.config.use_flash_attention = False

# Set model to evaluation mode
vl_gpt.eval()


MultiModalityCausalLM(
  (vision_model): CLIPVisionTower(
    (vision_tower): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Identity()
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in

In [19]:
conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\n Extract all the possible features in the bullet points for this image",
        "images": ["../AI_Genius_LinkedIn_Badge.png"],
    },
    {"role": "Assistant", "content": ""},
]

##/content/AI_Genius_LinkedIn_Badge.png

In [20]:
pil_images = load_pil_images(conversation)

prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

In [21]:
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)


RuntimeError: Input type (c10::BFloat16) and bias type (c10::Half) should be the same

In [11]:
inputs_embeds


tensor([[[ 0.0403, -0.0198, -0.7227,  ..., -0.0205,  0.0133,  0.0121],
         [ 0.1270, -0.0181, -0.1592,  ...,  0.2393, -0.0952,  0.0175],
         [-0.0271, -0.1836, -0.0913,  ..., -0.1172, -0.1216, -0.1377],
         ...,
         [ 0.2637, -0.0197, -0.0498,  ...,  0.0149, -0.0175, -0.0330],
         [-0.0327, -0.0063, -0.0767,  ...,  0.0223, -0.0825,  0.0332],
         [ 0.0991,  0.0776, -0.1494,  ...,  0.1289, -0.0864, -0.0123]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<IndexPutBackward0>)

In [13]:
import os
os.environ["FLASHATTN_FORCE_DISABLE"] = "1"


In [15]:
vl_gpt.language_model.config.use_flash_attention_2 = False
vl_gpt.language_model.config.use_flash_attention = False


In [16]:
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

RuntimeError: FlashAttention only supports Ampere GPUs or newer.

In [4]:
vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True
)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>\nConvert the formula into latex code.",
        "images": ["images/equation.png"],
    },
    {"role": "Assistant", "content": ""},
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation, images=pil_images, force_batchify=True
).to(vl_gpt.device)

# # run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# # run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True,
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


Some kwargs in processor config are unused and will not have any effect: add_special_token, sft_format, num_image_tokens, ignore_id, mask_prompt, image_tag. 


Add image tag = <image_placeholder> to the tokenizer


RuntimeError: FlashAttention only supports Ampere GPUs or newer.

In [25]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Disable FlashAttention explicitly (T4 does not support it)
os.environ["FLASHATTN_FORCE_DISABLE"] = "1"

# Define model path
#model_name = "malllabiisc/janus-1"
model_name = "deepseek-ai/Janus-1.3B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with correct settings for T4
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,  # Janus needs this for custom code
    torch_dtype=torch.bfloat16,  # T4 handles bfloat16 well
    device_map="cuda",  # Force model onto GPU
)

# Ensure FlashAttention is disabled if it exists in config
if hasattr(model.config, "use_flash_attention_2"):
    model.config.use_flash_attention_2 = False
if hasattr(model.config, "use_flash_attention"):
    model.config.use_flash_attention = False

# Set model to evaluation mode
model.eval()


MultiModalityCausalLM(
  (vision_model): CLIPVisionTower(
    (vision_tower): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Identity()
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in

In [27]:
# Define prompt
prompt = "What is the capital of France?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move input to GPU

# Generate response using forward pass
with torch.no_grad():
    output = model(**inputs)

# Decode response (may need adjustment based on model output structure)
response = tokenizer.decode(output.logits.argmax(dim=-1)[0], skip_special_tokens=True)
print(response)

TypeError: _forward_unimplemented() got an unexpected keyword argument 'input_ids'